# Cable Entry Orientations

Certainly! Here's an example in ArcPy that demonstrates creating a buffer around points, finding intersections with another feature layer, and adding attributes to the intersecting points:


In [1]:
import arcpy
import math
import arcpy
import os
import pandas as pd

In [2]:
# Set up your workspace and paths
arcpy.env.workspace = r'\\WM20ocqu46ph01\TEMP\Andrea_Sulova\HKW\0_HKW_Playground\0_HKW_Playground.gdb'
points_layer = "HKW_VII_WTG_L09_236D_DA2_56T_UTM31N_v0"  # Replace with your points layer nam
cable_layer = "HKW_VII_EAC_L05_DA2_53T_IMDC_string_UTM31_v2"  # Replace with the layer to intersect

# Name of the new field to create
wtg_name = "NAME"
x = "X_ETRS_UTM31"
y = "Y_ETRS_UTM31"



## Create a buffer around the points

In [3]:
# Create a new output feature class with the selected fields
buffer_output= "Buffer_100"
arcpy.Buffer_analysis(points_layer, buffer_output, "100 Meters")


<Result '\\\\WM20ocqu46ph01\\TEMP\\Andrea_Sulova\\HKW\\0_HKW_Playground\\0_HKW_Playground.gdb\\Buffer_100'>

## Create a new attribute Angle_between  

In [4]:
new_field_name = "Angle_between"

# Check if the field already exists and delete it if found
existing_fields = [field.name for field in arcpy.ListFields(buffer_output)]
if new_field_name in existing_fields:
     arcpy.DeleteField_management(buffer_output, new_field_name)
        
# Add a new text field with 255 characters
arcpy.AddField_management(buffer_output, new_field_name, "TEXT", field_length=255)
# Calculate the new field using values from an existing field
arcpy.CalculateField_management(buffer_output, new_field_name, "!" + wtg_name + "!", "PYTHON")


<Result 'Buffer_100'>

## Join WTG names intersecting cables to Field: Angle_between 

In [5]:
# Find the 'Name' field in the field mappings and set its output field properties
field_mappings = arcpy.FieldMappings()

# Add fields from the join feature class
field_mappings.addTable(buffer_output)


# Loop through the fields in the field mappings
for field in field_mappings.fields:
    if field.name == new_field_name:
        # Find the field map index for the specified field
        field_index = field_mappings.findFieldMapIndex(field.name)
        if field_index != -1:
            # Get the field map for the specified field index
            field_map = field_mappings.getFieldMap(field_index)
            # Set the merge rule for the field map
            field_map.mergeRule = 'Join'
            field_map.joinDelimiter = ' - '
            # Replace the field map in the field mappings object
            field_mappings.replaceFieldMap(field_index, field_map)

# Set environments using arcpy.EnvManager as a context manager
arcpy.analysis.SpatialJoin(
        cable_layer,
        buffer_output,
        "Spatial_Join",
        join_operation="JOIN_ONE_TO_ONE",
        join_type="KEEP_ALL",
        field_mapping = field_mappings,
        match_option="WITHIN_A_DISTANCE",
        search_radius="30 Meters")


<Result '\\\\WM20ocqu46ph01\\TEMP\\Andrea_Sulova\\HKW\\0_HKW_Playground\\0_HKW_Playground.gdb\\Spatial_Join'>

# Intersections

In [7]:

arcpy.analysis.Intersect([buffer_output,"Spatial_Join",],"intersections","ALL",None,output_type="INPUT")


<Result '\\\\WM20ocqu46ph01\\TEMP\\Andrea_Sulova\\HKW\\0_HKW_Playground\\0_HKW_Playground.gdb\\intersections'>

## Rename and Select Fields into the new feature class

In [19]:
# Raname Fields nad 
new_name_start = "Start_point"
if not arcpy.ListFields("intersections", new_name_start):
    arcpy.AlterField_management("intersections","Name", new_name_start ,new_name_start)
    print('Changed')

new_name_end = "End_point"    
if not arcpy.ListFields("intersections", new_name_end):
    arcpy.AlterField_management("intersections","Angle_between_1",new_name_end, new_name_end )
    print('Changed')
    
# Iterate through each row in the feature class using an update cursor
with arcpy.da.UpdateCursor("intersections", ["Start_point", "End_point"]) as cursor:
    for row in cursor:
        # Access field values using indexes or field names
        split_values = row[1].split(' - ')
        if len(split_values) >= 2:
            # Update the fields accordingly
            if split_values[0] == row[0]:
                row[1] = split_values[1].strip()  # Modify the value
                cursor.updateRow(row)  # Update the row in the feature class
            if split_values[1] == row[0]:
                row[1] = split_values[0].strip()  # Modify the value
                cursor.updateRow(row)  # Update the row in the feature class

# List of fields to keep (change these to the actual field names you want to keep)
fields_to_keep = [new_name_start, new_name_end,x,y ]

# Create a field mappings object
field_mappings = arcpy.FieldMappings()

# Iterate through all fields in the input feature class
for field in arcpy.ListFields("intersections"):
    # Add the field if it's in the list of fields to keep
    if field.name in fields_to_keep:
        # Create a field map for the selected field
        field_map = arcpy.FieldMap()
        field_map.addInputField("intersections", field.name)
        
        # Create an output field to the field mapping
        output_field = field_map.outputField
        output_field.name = field.name
        field_map.outputField = output_field

        # Add the field map to the field mappings object
        field_mappings.addFieldMap(field_map)

# Use FeatureClassToFeatureClass_conversion with field mapping to copy the selected fields
arcpy.FeatureClassToFeatureClass_conversion("intersections", arcpy.env.workspace, 
                                           arcpy.ValidateTableName("test", arcpy.env.workspace), 
                                           field_mapping=field_mappings)


Changed
Changed


<Result '\\\\WM20ocqu46ph01\\TEMP\\Andrea_Sulova\\HKW\\0_HKW_Playground\\0_HKW_Playground.gdb\\test'>

# Retrieve start and end points of the line

In [20]:

# Path to your feature class
feature_class = "test"

# Add new float fields to the feature class
arcpy.AddField_management(feature_class, "X1", "Text")
arcpy.AddField_management(feature_class, "Y1", "Text")
arcpy.AddField_management(feature_class, "X2", "Text")
arcpy.AddField_management(feature_class, "Y2", "Text")
arcpy.AddField_management(feature_class, "AngleFromNorth", "Double")  # Add a field for storing the angle from North

# Create an update cursor for your feature class
with arcpy.da.UpdateCursor(feature_class, ["SHAPE@", "X1", "Y1", "X2", "Y2", "AngleFromNorth", x, y]) as cursor:
    for row in cursor:
        # Access geometry of the line
        line_geometry = row[0]
        # Retrieve start and end points of the line
        start_point = line_geometry.firstPoint
        end_point = line_geometry.lastPoint
        
        if abs(start_point.X - row[6]) <= 10.0 and abs(start_point.Y - row[7]) <=10.0:
            row[1] = start_point.X 
            row[2] = start_point.Y 
            row[3] = end_point.X 
            row[4] = end_point.Y
            
            # Calculate the angle with respect to North
            angle_from_north = math.degrees(math.atan2(end_point.X - start_point.X, end_point.Y - start_point.Y))
            
            # Adjust the angle to be in the range [0, 360]
            angle_from_north = (angle_from_north + 360) % 360
            row[5] = angle_from_north  # Set the angle value in the "AngleFromNorth" field
            cursor.updateRow(row)
        else:
            row[1] = end_point.X
            row[2] = end_point.Y
            row[3] = start_point.X 
            row[4] = start_point.Y  
            angle_from_north = math.degrees(math.atan2(start_point.X - end_point.X, start_point.Y - end_point.Y))
            
            # Adjust the angle to be in the range [0, 360]
            angle_from_north = (angle_from_north + 360) % 360
            row[5] = angle_from_north  # Set the angle value in the "AngleFromNorth" field
            cursor.updateRow(row)


# List of fields to be removed
if arcpy.ListFields(feature_class, x):
    fields_to_remove = [x,y, "Shape_lenght"]
    arcpy.management.DeleteField(feature_class, fields_to_remove)



## Export a shapefile and an Excel file 

In [21]:
# Set the workspace (folder) where the shapefile and Excel file will be saved
output_folder = r"\\WM20ocqu46ph01\WF_Projects\NL_HKW\4_OUTPUT\20240125_GRID_Cable_Entry_Orientation_IMDC_Christian"
arcpy.env.workspace = output_folder

output_shapefile = os.path.join(output_folder, "Orientation_Cable.shp")
output_excel = os.path.join(output_folder, "Angle_Entry_Exit_Cable.xlsx")

# Export the shapefile and Excel
arcpy.CopyFeatures_management(feature_class, output_shapefile)
arcpy.TableToExcel_conversion(feature_class, output_excel)

print("Export completed successfully.")

Export completed successfully.
